In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/manufacturing-quality-control/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/manufacturing-quality-control/sample_submission.csv
/kaggle/input/manufacturing-quality-control/train.csv
/kaggle/input/manufacturing-quality-control/test.csv
/kaggle/input/manufacturing-quality-control/test/test_64.jpg
/kaggle/input/manufacturing-quality-control/test/test_42.jpg
/kaggle/input/manufacturing-quality-control/test/test_5.jpg
/kaggle/input/manufacturing-quality-control/test/test_34.jpg
/kaggle/input/manufacturing-quality-control/test/test_25.jpg
/kaggle/input/manufacturing-quality-control/test/test_22.jpg
/kaggle/input/manufacturing-quality-control/test/test_59.jpg
/kaggle/input/manufacturing-quality-control/test/test_23.jpg
/kaggle/input/manufacturing-quality-control/test/test_7.jpg
/kaggle/input/manufacturing-quality-control/test/test_28.jpg
/kaggle/input/manufacturing-quality-control/test/test_17.jpg
/kaggle/input/manufacturing-quality-control/test/test_49.jpg
/kaggle/input/manufacturing-quality-control/test/test_9.jpg
/kaggle/input/manufacturing-quality-c

In [18]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path 
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [19]:
# Configurações globais
altu = 288
larg = 186

labels = [
    "CONTENT_HIGH",
    "CONTENT_LOW",
    "COVER_NONE",
    "BOTTLE_SMASHED",
    "LABEL_WHITE",
    "LABEL_MISPLACED",
    "LABEL_NONE",
    "BOTTLE_NONE"
]


In [20]:
def detect_edges_refined_thin_lines(img_rgb):

    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    smooth = cv2.bilateralFilter(gray, d=9, sigmaColor=75, sigmaSpace=75)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(smooth)

    edges = cv2.Canny(enhanced, threshold1=50, threshold2=150)

    kernel_dilate = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    edges_dilated = cv2.dilate(edges, kernel_dilate, iterations=1)

    kernel_erode = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2,2))
    edges_thin = cv2.erode(edges_dilated, kernel_erode, iterations=1)

    edges_median = cv2.medianBlur(edges_thin, 3) 

    edges_smooth = cv2.GaussianBlur(edges_median, (5,5), sigmaX=1)

    _, edges_final = cv2.threshold(edges_smooth, 50, 255, cv2.THRESH_BINARY)

    return edges_final

In [21]:

def find_bottle_boxes_by_vertical_stripes(edge_mask, min_area=200):
    def binarize_mask(mask):
        if len(mask.shape) == 3:
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        _, bin_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        return bin_mask

    edge_bin = binarize_mask(edge_mask)
    h, w = edge_bin.shape
    stripe_width = w // 3

    bounding_boxes = []
    contours_all = []

    img_contours = cv2.cvtColor(edge_bin, cv2.COLOR_GRAY2RGB)

    for i in range(3):
        x_start = i * stripe_width
        x_end = (i + 1) * stripe_width if i < 2 else w

        stripe_mask = np.zeros_like(edge_bin)
        stripe_mask[:, x_start:x_end] = edge_bin[:, x_start:x_end]

        contours, _ = cv2.findContours(stripe_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = [c for c in contours if cv2.contourArea(c) > min_area]

        if contours:
            all_points = np.vstack(contours)
            x, y, w_box, _ = cv2.boundingRect(all_points)
            bounding_boxes.append((x, 0, w_box, h))
            contours_all.extend(contours)
            cv2.rectangle(img_contours, (x, 0), (x + w_box, h), (0, 255, 0), 2)
        else:
            bounding_boxes.append((x_start, 0, x_end - x_start, h))
            cv2.rectangle(img_contours, (x_start, 0), (x_end, h), (0, 255, 0), 2)

    return bounding_boxes, contours_all, img_contours


In [22]:
def tratamento_imagem(img):

    edges_thin = detect_edges_refined_thin_lines(img)


    bounding_boxes, contours_all, img_result = find_bottle_boxes_by_vertical_stripes(edges_thin, min_area=200)
    img_height, img_width = edges_thin.shape[:2]
    x, y, w, h = bounding_boxes[1]
    margin = int(0.10 * img_width)
    x_expanded = max(0, x - margin)
    w_expanded = min(img_width - x_expanded, w + 2 * margin)
    roi_center_expanded = edges_thin[y:y + h, x_expanded:x_expanded + w_expanded]
    roi_color_expanded = img[y:y + h, x_expanded:x_expanded + w_expanded]

    return cv2.cvtColor(roi_color_expanded, cv2.COLOR_RGB2BGR) 



In [23]:
def carregar_imagens(img_dir, ids=None, df_labels=None):
    imagens = []
    rotulos = []
    nomes_arquivos = []

    if ids is None:
        ids = sorted([f for f in os.listdir(img_dir) if f.endswith('.jpg')])

    for file_id in ids:
        img_path = os.path.join(img_dir, file_id)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = tratamento_imagem(img)
        img = cv2.resize(img, (larg, altu))
        img = img / 255.0
        
        imagens.append(img)
        nomes_arquivos.append(file_id)
        
        if df_labels is not None:
            rotulos.append(df_labels[df_labels['id'] == file_id].iloc[0, 1:].values.astype(np.float32))

    if df_labels is not None:
        return np.array(imagens, dtype=np.float32), np.array(rotulos, dtype=np.float32)
    else:
        return np.array(imagens, dtype=np.float32), nomes_arquivos


In [24]:
df_labels = pd.read_csv("/kaggle/input/manufacturing-quality-control/train.csv")

df_labels.fillna(0.0, inplace=True)


X, y = carregar_imagens("/kaggle/input/manufacturing-quality-control/train", ids=df_labels['id'].values, df_labels=df_labels)


In [25]:
# Dividir os dados em treino e validação (80% treino, 20% validação)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Rede neural convolucional
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(altu, larg, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='sigmoid') 
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 286, 184, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 143, 92, 32)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 141, 90, 64)         │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 70, 45, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 68, 43, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 34, 21, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 91392)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │      11,698,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,792,584 (44.99 MB)

 Trainable params: 11,792,584 (44.99 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,              
    batch_size=8,
    verbose=1
)

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 569ms/step - accuracy: 0.1318 - loss: 0.5615 - val_accuracy: 0.6875 - val_loss: 0.3390
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 526ms/step - accuracy: 0.2633 - loss: 0.4213 - val_accuracy: 0.6875 - val_loss: 0.3226
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 541ms/step - accuracy: 0.4981 - loss: 0.3517 - val_accuracy: 0.6875 - val_loss: 0.3369
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 508ms/step - accuracy: 0.3245 - loss: 0.3877 - val_accuracy: 0.6875 - val_loss: 0.3324
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 513ms/step - accuracy: 0.2560 - loss: 0.3550 - val_accuracy: 0.6875 - val_loss: 0.3034
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 523ms/step - accuracy: 0.3910 - loss: 0.3556 - val_accuracy: 0.6875 - val_loss: 0.2953
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 541ms/step - accuracy: 0.3604 - loss: 0.3625 - val_accuracy: 0.6875 - val_loss: 0.2742
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 593ms/step - accuracy: 0.4682 - loss: 0.3008 - val_accuracy: 0.6875 - val_loss:

In [27]:
from sklearn.metrics import classification_report

y_pred_prob = model.predict(X_val)
threshold = 0.5
y_pred = (y_pred_prob >= threshold).astype(int)

print(classification_report(y_val, y_pred, target_names=[
    "CONTENT_HIGH",
    "CONTENT_LOW",
    "COVER_NONE",
    "BOTTLE_SMASHED",
    "LABEL_WHITE",
    "LABEL_MISPLACED",
    "LABEL_NONE",
    "BOTTLE_NONE"
]))

accuracy = np.mean(np.all(y_val == y_pred, axis=1))
print(f"Acurácia exata (multi-label): {accuracy * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step
                 precision    recall  f1-score   support

   CONTENT_HIGH       1.00      0.71      0.83         7
    CONTENT_LOW       1.00      1.00      1.00         1
     COVER_NONE       1.00      1.00      1.00         1
 BOTTLE_SMASHED       0.00      0.00      0.00         0
    LABEL_WHITE       1.00      1.00      1.00         1
LABEL_MISPLACED       1.00      1.00      1.00         2
     LABEL_NONE       1.00      1.00      1.00         2
    BOTTLE_NONE       1.00      1.00      1.00         2

      micro avg       1.00      0.88      0.93        16
      macro avg       0.88      0.84      0.85        16
   weighted avg       1.00      0.88      0.93        16
    samples avg       0.69      0.66      0.67        16

Acurácia exata (multi-label): 87.50%


TESTE###########################

In [29]:

submission_df = pd.read_csv("/kaggle/input/manufacturing-quality-control/sample_submission.csv")

X_test, nomes_testes = carregar_imagens("/kaggle/input/manufacturing-quality-control/test")


y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)


submission_df.set_index("id", inplace=True)
for nome, pred in zip(nomes_testes, y_pred):
    submission_df.loc[nome] = pred  

submission_df.reset_index(inplace=True)


submission_df.to_csv("submission.csv", index=False)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 589ms/step
